[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XLM-RoBERTa.ipynb)

## Import XLM-RoBERTa models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for XLM-RoBERTa from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use XLM-RoBERTa models trained/fine-tuned on a specific task such as token/sequence classification.
- Some of the `XLM-RoBERTa` are larger than 2GB which is the limit in Spark NLP. Unfortunately, the models must be less than 2GB in size

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.6.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLMRobertaTokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q transformers==4.6.1 tensorflow==2.4.1 sentencepiece

     |████████████████████████████████| 2.3MB 5.1MB/s 
     |████████████████████████████████| 394.3MB 36kB/s 
     |████████████████████████████████| 1.2MB 32.6MB/s 
     |████████████████████████████████| 901kB 35.4MB/s 
     |████████████████████████████████| 3.3MB 31.8MB/s 
     |████████████████████████████████| 3.8MB 21.3MB/s 
     |████████████████████████████████| 471kB 59.5MB/s 
     |████████████████████████████████| 2.9MB 31.7MB/s 


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [xlm-roberta-base](https://huggingface.co/xlm-roberta-base) model from HuggingFace as an example
- In addition to `TFXLMRobertaModel` we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- Since `xlm-roberta-base` model is PyTorch we will use `from_pt=True` param to convert it to TensorFlow

In [ ]:
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel
import tensorflow as tf

# xlm-roberta-base
MODEL_NAME = 'xlm-roberta-base'

XLMRobertaTokenizer.from_pretrained(MODEL_NAME, return_tensors="pt").save_pretrained("./{}_tokenizer".format(MODEL_NAME))
TFXLMRobertaModel.from_pretrained(MODEL_NAME, from_pt=True).save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./xlm-roberta-base/saved_model/1/assets


INFO:tensorflow:Assets written to: ./xlm-roberta-base/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 1086380
-rw-r--r-- 1 root root        677 Jun  1 10:23 config.json
drwxr-xr-x 3 root root       4096 Jun  1 10:23 saved_model
-rw-r--r-- 1 root root 1112437880 Jun  1 10:23 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 7944
drwxr-xr-x 2 root root    4096 Jun  1 10:23 assets
-rw-r--r-- 1 root root 8123253 Jun  1 10:23 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun  1 10:23 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 4960
-rw-r--r-- 1 root root 5069051 Jun  1 10:21 sentencepiece.bpe.model
-rw-r--r-- 1 root root     238 Jun  1 10:21 special_tokens_map.json
-rw-r--r-- 1 root root     592 Jun  1 10:21 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `sentencepiece.bpe.model` file from the tokenizer
- all we need is to copy `sentencepiece.bpe.model` file into `saved_model/1/assets` which Spark NLP will look for

In [ ]:
# let's copy both vocab.txt file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {MODEL_NAME}/saved_model/1/assets

## Import and Save XLM-RoBERTa in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlmRoBertaEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.


In [ ]:
# entirely ignore this part
# it's for Colab only
! apt-get purge -y openjdk-11* -qq > /dev/null && sudo apt-get autoremove -y -qq > /dev/null
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! wget -q "https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz" > /dev/null
! tar -xvf spark-3.1.1-bin-hadoop2.7.tgz > /dev/null
! SPARKHOME="/content/spark-3.1.1-bin-hadoop2.7"
! export SPARK_HOME=$SPARKHOME
! export JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

# Install pyspark spark-nlp
! pip install --upgrade -q pyspark==3.1.1 findspark
! pip install --upgrade --force-reinstall https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/spark-31-dev/spark_nlp-3.1.0-py2.py3-none-any.whl

from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName("Spark NLP")\
.master("local[*]")\
.config("spark.driver.memory","16G")\
.config("spark.driver.maxResultSize", "0") \
.config("spark.kryoserializer.buffer.max", "2000M")\
.config("spark.jars", "https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/spark-31-dev/spark-nlp-assembly-3.1.0.jar")\
.getOrCreate()

     |████████████████████████████████| 212.3MB 53kB/s 
     |████████████████████████████████| 204kB 10.7MB/s 
     |████████████████████████████████| 51kB 1.6MB/s 


In [ ]:
from sparknlp.annotator import *

xlm_roberta = XlmRoBertaEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setStorageRef('xlm_roberta_base') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
xlm_roberta.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your XLM-RoBERTa model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 288996
drwxr-xr-x 5 root root      4096 May 31 16:07 fields
drwxr-xr-x 2 root root      4096 May 31 16:07 metadata
-rw-r--r-- 1 root root 295923287 May 31 16:13 roberta_tensorflow


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊 

In [ ]:
xlm_roberta_loaded = XlmRoBertaEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(True)

In [ ]:
roberta_loaded.getStorageRef()

'roberta_base'

That's it! You can now go wild and use hundreds of XLM-RoBERTa models from HuggingFace 🤗 in Spark NLP 🚀 
